We have a dataset (DCC) with a shortage of certain labels. We want to generate 
new samples synthetically using GPT-4. We will use the following approach:
1. We take the existing samples for each document type and present these to GPT-4
2. we ask to generate new sentences like it, where the token labels are provided in the BIO format

We care specifically about the following labels:
* Experiencer: Other
* Historical: Hypothetical

The task of the GPT model is to generate new sentences that are similar to the input sentences but with variations of the medical concepts. 




# Definitions
The definitions are taken from the ConText/ConTextD papers:

## Negation

This property has two values, ‘Negated’ or ‘Not negated’. A clinical condition or term is labeled as ‘Negated’ if there is evidence in the text suggesting that the condition does not occur or exist, e.g., ‘There was no sign of sinus infection’, otherwise it is ‘Not negated’.

## Temporality

The temporality property places a condition along a time line. There are three possible values for this property: ‘Recent’, ‘Historical’, and ‘Hypothetical’. A condition is considered ‘Recent’ if it is maximally 2 weeks old. Conditions that developed more than 2 weeks ago are labeled as ‘Historical’. A condition is labeled as ‘Hypothetical’ if it is not ‘Recent’ or ‘Historical’, e.g., ‘patient should return if she develops fever’ [13].

**Adaptation**: *'Hypothetical' is specifically about (theoretical) concepts, concepts that are not (yet) realized, i.e. concepts that may materialize in the future. 'Historical' and 'Recent' can be used for realized concepts, in which we also include their negations. I.e. if a concept is explicitly denied historically or recently, we can label it as 'Historical' or 'Recent' respectively.*

## Experiencer

Clinical text may refer to subjects other than the actual patient. The experiencer property describes whether the patient experienced the condition or someone else. For simplicity, we have defined only two possible values for this property: ‘Patient’ or ‘Other’, where ‘Other’ refers to anyone but the actual patient, e.g., ‘Mother is recently diagnosed with cancer’.

In [365]:
%load_ext autoreload
%autoreload 2

import os, sys, re
import json, dotenv
import pprint

import openai
import asyncio
from openai import AsyncOpenAI, OpenAI
from tqdm import tqdm
from collections import defaultdict

import pandas as pd

import datetime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import preprocess_dcc_for_robbert
from utils import active_synthesis
from utils import synthesis_prompts

dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [ ]:
run_temporality = True
run_experiencer = True
#run_historical = True
experiencer_file = '../data/synth_experiencer_gpt_4_1106_preview_20231214.parquet'
hypothetical_file = '../data/synth_temporality_gpt_4_1106_preview_20231218.parquet'

In [ ]:
openai.api_key = os.getenv("OPENAI_KEY")

In [ ]:
DCC = json.load(open('../data/emc-dcc_ann.json'))

In [ ]:
update_dcc = DCC.copy()
for c in update_dcc['projects'][0]['documents']:
    c['source'] = 'EMC_DCC_ORIGINAL'
with open('../data/emc-dcc_ann_ORIGNAL.json', 'w') as f:
    json.dump(update_dcc, f, indent=2)

In [ ]:
docs = update_dcc['projects'][0]['documents']

In [285]:
def get_class_counts(docs: dict):
    class_counts = {'Negation': defaultdict(int),
                'Temporality': defaultdict(int),
                'Experiencer': defaultdict(int)}

    for doc in docs:
        for ann in doc['annotations']:
            for _class, val in ann['meta_anns'].items():
                class_counts[_class][val['value']] += 1
    return class_counts

In [301]:
pprint.pprint(get_class_counts(docs), indent=2)

{ 'Experiencer': defaultdict(<class 'int'>,
                             { None: 5,
                               'other': 97,
                               'patient': 12449}),
  'Negation': defaultdict(<class 'int'>,
                          { 'negated': 1760,
                            'not negated': 10791}),
  'Temporality': defaultdict(<class 'int'>,
                             { 'historical': 521,
                               'hypothetical': 98,
                               'recent': 11932})}


In [ ]:
#len(relevant_docs_hypothetical), len(relevant_docs_experiencer)
#relevant_docs_hypothetical[0]['text'][0:110]
#[(d['start'],d['end'], d['id']) for d in relevant_docs_hypothetical[0]['annotations'] 
#        if d['meta_anns']['Temporality']['value']=='hypothetical']
#relevant_docs_hypothetical[0]['name']

In [292]:
Correction_of_original = active_synthesis.Annotation_correction_original
Correction_of_synthetic = active_synthesis.Annotation_correction_synthetic

In [293]:
# update the docs
update_docs = docs.copy()
for c in Correction_of_original:
    for d in update_docs:        
        if d['name']==c['doc_id']:
            d['source'] = 'EMC_DCC_ORIGINAL_ADJUSTED'
            for a in d['annotations']:
                if a['id']==c['annotation_id']:
                    a['meta_anns'][c['meta']]['value'] = c['value']
# put updated docs in DCC
DCC['projects'][0]['documents'] = update_docs

In [294]:
pprint.pprint(get_class_counts(update_docs), indent=2)

{ 'Experiencer': defaultdict(<class 'int'>,
                             { None: 5,
                               'other': 97,
                               'patient': 12449}),
  'Negation': defaultdict(<class 'int'>,
                          { 'negated': 1760,
                            'not negated': 10791}),
  'Temporality': defaultdict(<class 'int'>,
                             { 'historical': 521,
                               'hypothetical': 98,
                               'recent': 11932})}


In [ ]:
# write DCC back to json 
with open('../data/emc-dcc_ann_ADJ.json', 'w') as f:
    json.dump(DCC, f, indent=2)

In [362]:
# minority classes
relevant_docs_hypothetical = []
for i, doc in enumerate(update_docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Temporality']['value']=='hypothetical'):
            doc['index'] = i
            relevant_docs_hypothetical.append(doc)
            break
        
relevant_docs_other = []
for i, doc in enumerate(update_docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Experiencer']['value']=='other'):
            doc['index'] = i
            relevant_docs_other.append(doc)
            break
        
relevant_docs_historical = []
for i, doc in enumerate(update_docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Temporality']['value']=='historical'):
            doc['index'] = i
            relevant_docs_historical.append(doc)
            break

relevant_docs_patient_names = [d.split("_")[0] for d in active_synthesis.Experiencer_patient_ASL1]

relevant_docs_patient = []

for i, doc in enumerate(update_docs):
    if doc['name'] in relevant_docs_patient_names:
        relevant_docs_patient.append(doc)


In [ ]:
OAI_ASYNC_CLIENT = AsyncOpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)
OAI_CLIENT = OpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)

In [367]:
SYSTEM_PROMPT_HYPOTHETICAL = synthesis_prompts.SYSTEM_PROMPT_HYPOTHETICAL
SYSTEM_PROMPT_HYPOTHETICAL_CHECK = synthesis_prompts.SYSTEM_PROMPT_HYPOTHETICAL_CHECK

SYSTEM_PROMPT_EXPERIENCER = synthesis_prompts.SYSTEM_PROMPT_EXPERIENCER
SYSTEN_PROMPT_EXPERIENCER_CHECK = synthesis_prompts.SYSTEM_PROMPT_EXPERIENCER_CHECK

SYSTEM_PROMPT_HISTORICAL = synthesis_prompts.SYSTEM_PROMPT_HISTORICAL
SYSTEM_PROMPT_HISTORICAL_CHECK = synthesis_prompts.SYSTEM_PROMPT_HISTORICAL_CHECK

In [ ]:
def get_chat_res(USER_TEXT='Good day', 
                 SYSTEM_PROMPT="Please be kind in 20 years", 
                 n = 10,
                 MODEL="gpt-4"):
    return OAI_CLIENT.chat.completions.create(
            model=MODEL,
            n = n,
            temperature=0.,
            messages=[
                        {"role": "system",
                        "content": SYSTEM_PROMPT
                        },
                        {"role": "user", 
                        "content": USER_TEXT
                        }],
            stream=False,
        )

In [ ]:
GPT_VERSION = 'gpt-4-1106-preview'
CURRENT_DATE = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
if run_temporality:
    nieuwe_zinnen_hypothetisch = []
    for i, doc in tqdm(enumerate(relevant_docs_hypothetical)):
        EXAMPLE = doc['text'].replace('|', ' ')
        # add | vertical bars around the concept that needs to be replaced
        LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                    if d['meta_anns']['Temporality']['value']=='hypothetical']
        for loc in LOCS:
            EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
        
        res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                        n=10,
                        MODEL=GPT_VERSION, # gpt-3.5-turbo-instruct-0914
                        USER_TEXT="VOORBEELDTEKST: " + EXAMPLE)

        for j, _res in enumerate(res.choices):
            txt = _res.message.content
            nieuwe_zinnen_hypothetisch.append((
                doc['name'],
                'hypothetical',
                j,
                txt[txt.find('NIEUWE_TEKST')+12:].strip()))    
    
    hypothetical_df = pd.DataFrame(nieuwe_zinnen_hypothetisch, columns=['doc_id', 'class_value', 'synth_num', 'text'])
    hypothetical_df.to_parquet(f'../data/synth_temporality_{GPT_VERSION.replace("-", "_")}_{CURRENT_DATE}.parquet')
else:
    hypothetical_df = pd.read_parquet(hypothetical_file)
    hypothetical_df['class_value'] = 'hypothetical'
    hypothetical_df = hypothetical_df.assign(text=hypothetical_df.text.str.lstrip(to_strip=':'))

In [ ]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
if run_experiencer:
    nieuwe_zinnen_experiencer = []
    for i, doc in tqdm(enumerate(relevant_docs_other)):
        EXAMPLE = doc['text'].replace('|', ' ')
        # add | vertical bars around the concept that needs to be replaced
        LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                    if d['meta_anns']['Experiencer']['value']=='other']
        for loc in LOCS:
            EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
        
        res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_EXPERIENCER, 
                        n=10,
                        MODEL=GPT_VERSION, # gpt-3.5-turbo-instruct-0914
                        USER_TEXT="VOORBEELDTEKST: " + EXAMPLE)

        for j, _res in enumerate(res.choices):
            txt = _res.message.content
            nieuwe_zinnen_experiencer.append((
                doc['name'],
                'other',
                j,
                txt[txt.find('NIEUWE_TEKST')+12:].strip()))
    experiencer_df = pd.DataFrame(nieuwe_zinnen_experiencer, columns=['doc_id', 'class_value', 'synth_num', 'text'])
    experiencer_df.to_parquet(f'../data/synth_experiencer_{GPT_VERSION.replace("-", "_")}_{CURRENT_DATE}.parquet')
else:
    experiencer_df = pd.read_parquet(experiencer_file)
    experiencer_df['class_value'] = 'other'


In [ ]:
# run the checks
def check_by_gpt(TXT: str='', 
                 N_checks: int=7,
                 SYSTEM_PROMPT: str="Respect humans"
                 )->str:
    N_checks = 7
    maj_vote = N_checks//2

    RES = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT,
                        USER_TEXT="VOORBEELDTEKST: " + TXT,
                        n=N_checks,
                        MODEL=GPT_VERSION)

    RES_sel = []
    no_count = defaultdict(int)
    for j, _res in enumerate(RES.choices):
        txt = _res.message.content
        _d = eval(txt)
        RES_sel.append((j, _d))
        
        list_lens = []
        for k, v in _d.items():
            if v=='nee':
                no_count[k] += 1
        
        list_lens.append(len(_d.keys())) 

    if len(set(list_lens))>1:
        return 'ERROR-checker concept count mismatch'
    else:
        if len(no_count.values())>0:    
            # approach: if any of the concept is deemed incorrect we flag the TXT for removal 
            num_exc = sum([_v>maj_vote for _v in no_count.values()])
            if num_exc>0:
                if num_exc == len(_d.keys()):
                    return False
                else:
                    spans = []                    
                    for r in re.finditer(r'(\|.*?\|)', TXT):
                        spans.append(r.span())  
                    if len(spans)<max(_d.keys()):
                        return 'ERROR-checker concept count mismatch'              
                    rem = []
                    for k, v in no_count.items():
                        if v>maj_vote:
                            print(spans, no_count)
                            rem.append(spans[k])
                    if len(rem)>0:
                        for rcount, r in enumerate(rem):
                            TXT = TXT[:r[0]-rcount*2]+\
                                TXT[r[0]+1-rcount*2:r[1]-1-rcount*2]+\
                                TXT[r[1]-rcount*2:]
    return TXT
    

In [ ]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
texts_checked = [None]*experiencer_df.shape[0]
texts = experiencer_df['text'].values
for k, text in tqdm(enumerate(texts), total=len(texts)):
    if texts_checked[k] is None:
        texts_checked[k]=check_by_gpt(text, 
                                    N_checks=7, 
                                    SYSTEM_PROMPT=SYSTEM_PROMPT_EXPERIENCER_CHECK)
experiencer_df['checked_text'] = texts_checked

c1 = experiencer_df['checked_text']!=False 
c2 = experiencer_df['checked_text']!='ERROR-checker concept count mismatch'

experiencer_df = experiencer_df[c1 & c2]

# check token length
experiencer_df['token_len'] = experiencer_df['checked_text'].astype(str)\
                            .progress_apply(lambda x: len(x.split()))

# check number of r'\|.*?\|' in the text
experiencer_df['n_concepts'] = experiencer_df['checked_text'].astype(str)\
                            .progress_apply(lambda x: len(re.findall(r'\|.*?\|', x)))
                            
c = experiencer_df['n_concepts']>0
experiencer_df = experiencer_df[c]

experiencer_df = experiencer_df.drop(['text'], axis=1)
experiencer_df = experiencer_df.rename(columns={'checked_text': 'text'})
experiencer_df.to_parquet(f'../data/synth_experiencer_{GPT_VERSION.replace("-", "_")}_{CURRENT_DATE}.parquet')

In [ ]:
#texts_checked = [None]*hypothetical_df.shape[0]
texts = hypothetical_df['text'].values
for k, text in tqdm(enumerate(texts), total=len(texts)):
    if texts_checked[k] is None:
        texts_checked[k]=check_by_gpt(text, 
                                    N_checks=3, 
                                    SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL_CHECK)
hypothetical_df['checked_text'] = texts_checked

c1 = hypothetical_df['checked_text']!=False 
c2 = hypothetical_df['checked_text']!='ERROR-checker concept count mismatch'

hypothetical_df = hypothetical_df[c1 & c2]

# check token length
hypothetical_df['token_len'] = hypothetical_df['checked_text'].astype(str)\
                            .progress_apply(lambda x: len(x.split()))

# check number of r'\|.*?\|' in the text
hypothetical_df['n_concepts'] = hypothetical_df['checked_text'].astype(str)\
                            .progress_apply(lambda x: len(re.findall(r'\|.*?\|', x)))
                            
c = hypothetical_df['n_concepts']>0
hypothetical_df = hypothetical_df[c]

hypothetical_df = hypothetical_df.drop(['text'], axis=1)
hypothetical_df = hypothetical_df.rename(columns={'checked_text': 'text'})
hypothetical_df.to_parquet(f'../data/synth_hypothetical_{GPT_VERSION.replace("-", "_")}_{CURRENT_DATE}.parquet')

### Extract the spans from the synthetic set and add them to the original dataset with an additional label

In [274]:
def put_in_dict(data, original_documents, Class_name='Experiencer', Class_value=None):
    max_id = max([int(k['id']) for d in update_docs
                           for k in d['annotations'] if k['id'] is not None])
    
    new_documents = original_documents.copy()
    
    if type(data)==pd.DataFrame:
        data = [(r.doc_id, r.class_value, r.synth_num, r.text)  for r in data.itertuples()]
    
    for i, (name, Class_value_spec, subid, text) in enumerate(data): # start=max_id+1
        Class_value_spec = Class_value_spec if Class_value_spec is not None else Class_value
        
        if i == 0:
            true_i = i + max_id + 1
        else:
            true_i = true_i + 1
        
        clean_text = text.replace('|', '')
        _doc = {
            'id': true_i,
            'source': 'synthetic',
            'source_version': f"{GPT_VERSION}|{CURRENT_DATE}",     
            'name': f"{name}|synth|{Class_name}|{subid}|{CURRENT_DATE}",
            'text': clean_text,
            'annotations' : []
        }
        
        try:
            for concept_count, match in enumerate(re.finditer('\|[A-zÀ-ÿ\s]+\|', text)):
                start, end = match.span()
                start_clean = start + 1 -1-concept_count*2 #  (+1,-1)  (+1,-3), (+1,-5), (+1,-7)...
                end_clean = end - 1 -1-concept_count*2 # (-1,-1), (-1,-3), (-1,-5), (-1,-7)...
                true_i = true_i + 1
                _doc['annotations'].append(
                            {
                                'id': true_i,
                                'user': 'emc_dcc_synth',
                                'cui': 1,
                                'start': start_clean,
                                'end': end_clean,
                                'value': text[start+1:end-1],
                                'validated': False,
                                'correct': True,
                                'alternative': False,
                                'killed': False,
                                'meta_anns': {
                                    Class_name: {'value': Class_value_spec,
                                                    'name': Class_name,
                                                    'validated': False,
                                                    'acc': 1.0
                                                    },
                                }            
                            }
                        )         
        except:
            print(i, text)
        new_documents.append(_doc)
    return new_documents

In [ ]:
#experiencer_df = pd.read_parquet('../data/synth_experiencer_gpt_4_1106_preview_20231214.parquet')
#hypothetical_df = pd.read_parquet('../data/synth_temporality_gpt_4_1106_preview_20231218.parquet')

In [262]:
experiencer_df = experiencer_df[['doc_id', 'class_value', 'synth_num', 'text']]
hypothetical_df = hypothetical_df[['doc_id', 'class_value', 'synth_num', 'text']]

In [319]:
new_docs_experiencer = put_in_dict(experiencer_df, update_docs, 
                                   Class_name='Experiencer', 
                                   Class_value='other')
new_docs_temporality = put_in_dict(hypothetical_df, new_docs_experiencer,
                                   Class_name='Temporality', 
                                   Class_value='hypothetical')

In [317]:
get_class_counts(new_docs_temporality)

{'Negation': defaultdict(int, {'not negated': 10791, 'negated': 1760}),
 'Temporality': defaultdict(int,
             {'recent': 11932, 'historical': 521, 'hypothetical': 159}),
 'Experiencer': defaultdict(int, {'patient': 12449, 'other': 773, None: 5})}

In [320]:
new_docs_temporality_copy = new_docs_temporality.copy()
for c in Correction_of_synthetic:
    for d in new_docs_temporality_copy:        
        if c['doc_id'] in d['name']:
            d['source'] = 'synthetic_ADJUSTED'
            for a in d['annotations']:
                if a['start']==c['start']:
                    print(a['start'], c['start'])
                    a['meta_anns'][c['meta']]['value'] = c['value']
# put updated docs in DCC

150 150
143 143


In [322]:
max_id = max([int(k['id']) for d in update_docs
                           for k in d['annotations'] if k['id'] is not None])

print(f"Number of annotations in original DCC: {max_id}") 

max_id = max([int(k['id']) for d in new_docs_temporality_copy
                           for k in d['annotations'] if k['id'] is not None])

print(f"Number of annotations in synthetic augmented DCC: {max_id}")

Number of annotations in original DCC: 17915
Number of annotations in synthetic augmented DCC: 19219


## Write new samples to the dataset

In [323]:
DCC['projects'][0]['documents'] = new_docs_temporality_copy

In [324]:
# write DCC back to json 
with open('../data/emc-dcc_ann_Augmented_ASL1_Experiencer.json', 'w', encoding='utf-8') as f:
    json.dump(DCC, f, indent=2)

## Write to DCC_df

In [340]:
texts, labels, ids = preprocess_dcc_for_robbert.get_tuples_from_medcat_json('../data/emc-dcc_ann_Augmented_ASL1_Experiencer.json')

100%|██████████| 6045/6045 [00:00<00:00, 82810.71it/s]


In [344]:
dataset, errors = preprocess_dcc_for_robbert.get_dataset(texts, labels, ids)

# index error:
# only one label, label for word at the end

# Mismatch:
# \# preceding, part of compound

len(errors)

Index Error, in GP1650|synth|Experiencer|4|20240207, for line:['T1', 'Other 126 135', 'hemofilie']
Index Error, in GP1905|synth|Experiencer|1|20240207, for line:['T1', 'Other 162 191', 'chronische nierinsufficiëntie']
Index Error, in GP1905|synth|Experiencer|3|20240207, for line:['T1', 'Other 146 157', 'osteoporose']


25

In [345]:
df, ids = preprocess_dcc_for_robbert.get_dataframe(dataset)
print(f"\tProcessed {len(set(ids))} files")
df.to_csv("../data/DCC_df_ASL1_experiencer.csv", index=False, sep="\t")

	Processed 6045 files


In [347]:
df.Experiencer.value_counts()

Experiencer
O          184116
Patient     19676
Other        1425
Name: count, dtype: int64

In [348]:
df.Temporality.value_counts()

Temporality
O               185268
Recent           18860
Historical         822
Hypothetical       267
Name: count, dtype: int64

In [349]:
df.Negation.value_counts()

Negation
O             185378
NotNegated     16781
Negated         3058
Name: count, dtype: int64

## Translate English corpora

* English -- BioScope; [HF](https://huggingface.co/datasets/bigbio/bioscope), [src](https://rgai.inf.u-szeged.hu/downloads)
* English -- [Genia](http://www.geniaproject.org/genia-corpus/term-corpus)
* English -- Sherlock, SFU review corpus
